<h2>Trabalhando com Web - Protocolos http, APIs REST e Web Scraping</h2>

### Fundamentos do Protocolo http

Antes de escrever código, é crucial entender como a web "conversa".

* O Modelo Cliente-Servidor: Seu script R (cliente) faz um pedido (Request) a um servidor, e o servidor devolve uma resposta (Response).
* Métodos HTTP (Verbos):
    * GET: Pede dados (o mais comum em scraping).
    * POST: Envia dados (comum em formulários ou logins).
* Status Codes:
    * 200 OK: Sucesso.
    * 404 Not Found: A página não existe.
    * 403 Forbidden: Você não tem permissão (muito comum se o site bloquear scrapers).
    * 500 Internal Server Error: Erro no servidor.


Em R, utiliza-se a biblioteca `httr2`(padrão moderno) ou `httr`

In [ ]:
#install.packages(c("httr2", "jsonlite", "rvest", "tidyverse"))

library(httr2)
library(jsonlite)
library(dplyr)

# 1. Definir a URL (Endpoint)
url_api <- "https://economia.awesomeapi.com.br/last/USD-BRL"

# 2. Fazer a requisição
resposta <- request(url_api) |> # operador de conexão moderno
  req_perform()

# 3. Ler o conteúdo JSON
conteudo <- resp_body_json(resposta)

# 4. Acessar os dados (o JSON vira uma lista no R)
cotacao <- conteudo$USDBRL
print(paste("A cotação atual é:", cotacao$bid))

# Dica: Converter JSON direto para Data Frame muitas vezes é útil
# conteudo_df <- fromJSON(resp_body_string(resposta))

Aqui está a explicação detalhada de cada peça:
1. request(url_api): A Preparação (O Envelope)

A função request() cria o que chamamos de Objeto de Requisição.

* O que ela faz: Ela pega a URL (o endereço do site) e prepara um "pacote" virtual com as configurações básicas.
* O que ela NÃO faz: Ela não acessa a internet ainda. Se você rodar apenas request("http://site.com"), nada acontece na rede. O seu computador não enviou nada para o servidor.
* Analogia: É como escrever uma carta e colocar o endereço no envelope. A carta está pronta, mas ainda está na sua mão.


2. req_perform(): A Ação (O Envio)

A função req_perform() é quem realmente trabalha.

* O que ela faz: Ela pega aquele objeto de requisição que você preparou, conecta-se à internet, bate na porta do servidor (API), envia o pedido e fica esperando a resposta.
* O Resultado: Quando o servidor responde, o req_perform() captura tudo (o status 200, os cabeçalhos técnicos e os dados/JSON) e devolve isso para o R.
* Analogia: É o momento em que você coloca a carta na caixa do correio e espera o carteiro trazer a resposta.

Observação: Formato do Objeto de Requisição é uma Lista, pois R usa OOP: S3 (semelhante à OOP, mas objetos (listas) não possuem métodos próprios)

### Web Scraping

Quando não existe uma API, precisamos "raspar" o HTML da página, que é não-estruturado.

* HTML: A estrutura da página (tags como `<div>`, `<h1>`, `<table>`, `<a>`).
* CSS Selectors: A forma de apontar para elementos específicos na página (ex: .titulo pega elementos com class="titulo").

No R: O pacote rvest

O rvest é o padrão ouro para scraping em R. O fluxo geralmente é: `read_html()` -> `html_element()` -> `html_text()` ou `html_attr()`.


In [ ]:
library(rvest)

url_wiki <- "https://pt.wikipedia.org/wiki/R_(linguagem_de_programação)"

# 1. Ler o HTML da página
pagina <- read_html(url_wiki)

# 2. Extrair o título principal (h1)
# O seletor CSS para o título principal geralmente é "h1" ou "#firstHeading" na Wikipedia
titulo <- pagina |>
  html_element("#firstHeading") |>
  html_text()

print(titulo)

# 3. Extrair uma tabela da página
# O rvest tem uma função mágica para tabelas
tabela_info <- pagina |>
  html_element(".infobox") |> # Tenta pegar a caixa de informação
  html_table()

print(head(tabela_info))

Boas Práticas e Ética (Muito Importante!)

* Respeite o robots.txt: Verifique sempre site.com/robots.txt para saber se o site permite scraping. O pacote polite em R ajuda a fazer isso automaticamente.
* Não derrube o site: Coloque pausas entre as requisições (Sys.sleep(2)) se estiver fazendo um loop.
* User-Agent: Identifique-se. Sites bloqueiam requisições que não parecem navegadores ou que não têm identificação.

In [ ]:
request("https://site.com") |>
  req_user_agent("Meu Bot de Estudos (email@exemplo.com)")